# Criação dos datasets considerando os cursos de TI do Brasil e da Bahia

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_style('whitegrid')

In [ ]:
df_sup_aluno_tic = pd.read_csv('dataset/SUP_ALUNO_TIC_2015_2019.CSV')
df_curso_tic = pd.read_csv('dataset/SUP_CURSO_TIC_2015_2019.CSV', usecols=['NU_ANO_CENSO', 'CO_IES', 'CO_CURSO', 'NO_CURSO', 'CO_UF'])
df_dados_ies = pd.read_csv('dataset/SUP_IES_2015_2019.CSV', encoding='ISO-8859-1', usecols=['NU_ANO_CENSO', 'CO_IES', 'SG_IES', 'CO_UF'])


In [ ]:
df_sup_aluno_tic.head()

In [ ]:
df_curso_tic = df_curso_tic.fillna(0)
df_dados_ies = df_dados_ies.fillna(0)
df_curso_tic = df_curso_tic.astype({'CO_UF': 'int32'})

df_dados_ies = df_dados_ies.replace({'CO_UF':{' CULTURA E TECNOLOGIA SAO FIDELIS LTDA - EPP': 33}})
df_dados_ies = df_dados_ies.astype({'CO_UF': 'int32'})

In [ ]:
outer_merged = pd.merge(df_sup_aluno_tic, df_curso_tic, how="outer", on=['NU_ANO_CENSO', 'CO_IES', 'CO_CURSO'])

In [ ]:
outer_merged = pd.merge(outer_merged, df_dados_ies, how='outer', on=['NU_ANO_CENSO', 'CO_IES', 'CO_UF'])

## Filtros

Selecionando apenas os cursos da Bahia

In [ ]:
curso_aluno_BA = outer_merged.loc[outer_merged.CO_UF == 29]
display(curso_aluno_BA.shape)

Selecionando apenas os cursos presenciais

In [ ]:
display(curso_aluno_BA.TP_MODALIDADE_ENSINO.unique())
display(curso_aluno_BA.TP_MODALIDADE_ENSINO.value_counts())
curso_presencial = curso_aluno_BA.loc[curso_aluno_BA.TP_MODALIDADE_ENSINO == 1]
display(curso_presencial.shape)

Selecionando apenas os cursos de Universidades públicas

In [ ]:
curso_presencial  = curso_presencial .loc[curso_presencial .TP_CATEGORIA_ADMINISTRATIVA.isin([1, 2, 3])]
curso_presencial.shape

selecionando apenas os alunos cursando

In [ ]:
curso_presencial = curso_presencial.loc[curso_presencial['TP_SITUACAO'] == 2]
curso_presencial.groupby(['NO_CURSO']).size()

In [ ]:
curso_presencial.groupby(['CO_IES']).size()

In [ ]:
display(curso_presencial.shape)

In [ ]:
outer_merged.to_csv('dataset/alunos_curso_2015_2019_tic_br.csv', index=False)
curso_presencial.to_csv('dataset/alunos_curso_2015_2019_tic_BA.csv', index=False)